In [1]:
# NOTEBOOK FOR UNDERSAMPLING STRATEGIES
# remove all the consecutive duplicate clicks in a sample
# remove all the duplicate rows of clicks in the entire df  
#todo fix timedelta. right now the transformation for the time delta is ignored.

In [19]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
text_num_mapping_start = 1
model_test_run = False #This reduces the data generated by selecting significantly lesser number of sessions to test the model
remove_consecutive_identical_clicks = False
#Todo implement identical consecutive clicks - only do that if the timedelta between clicks is quite less- otherwise it doesnt make sense.
sequence_augmentation = True
carsi_labels_only = True
regenerate_context_data = False
whole_session_context = False
data_autoencoder = False
pad_to_window_size = True
window = 100 #seconds

base_path = '../datasets/sequential/'
combined_context_path = '../data/05_Interaction_Sequences/context.csv'
augmentation_folder = 'aug/' if sequence_augmentation else 'non_aug/'
if model_test_run:
    augmentation_folder = 'test/aug/' if sequence_augmentation else 'test/non_aug/'
sequence_context_path = f'{base_path}{augmentation_folder}parameters/sequence_context.csv'
parameter_path = f'{base_path}{augmentation_folder}parameters'
sequence_path = f'{base_path}{augmentation_folder}seq'
train_session_path = f'{base_path}{augmentation_folder}parameters/train_sessions.pkl'
test_session_path = f'{base_path}{augmentation_folder}parameters/test_sessions.pkl'
train_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/train.csv'
test_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/test.csv'
train_static_context_path = f'{base_path}{augmentation_folder}static_context/train.csv'
test_static_context_path = f'{base_path}{augmentation_folder}static_context/test.csv'

dc_train_unnor_path = '../datasets/sequential/aug/dynamic_context/unnormal/train.csv'
dc_test_unnor_path = '../datasets/sequential/aug/dynamic_context/unnormal/test.csv'

In [4]:
# dc_train = pd.read_csv(train_dynamic_context_path)
# dc_test = pd.read_csv(test_dynamic_context_path)

dc_train = pd.read_csv(dc_train_unnor_path)
dc_test = pd.read_csv(dc_test_unnor_path)

sc_train = pd.read_csv(train_static_context_path)
sc_test = pd.read_csv(test_static_context_path)

seq_train = pd.read_csv(sequence_path+'/train.tsv', sep='\t')
seq_test = pd.read_csv(sequence_path+'/test.tsv', sep='\t')

In [5]:
def remove_consecutive_duplicates(input_list):
    result = []
    removed_indices = []  # List to store indices of removed elements
    prev = None
    for i, num in enumerate(input_list):
        if num != prev:
            result.append(num)
        else:
            removed_indices.append(i)
        prev = num
    return result, removed_indices

def join_list(lst):
    return ' '.join(map(str, lst))

def create_zeros_list(lst):
    return [0] * len(lst)
    
def seq_duplicate_remove(df, datasettype):
    df['interaction_time_delta_train_list'] = df['interaction_time_delta_train'].str.split().apply(lambda x: [int(i) for i in x])
    result = df['item_id_seq_train'].str.split().apply(lambda x: remove_consecutive_duplicates([int(i) for i in x]))
    # Extract cleaned list and removed indices into separate columns
    df['seq_list'] = result.apply(lambda x: x[0])
    df['removed_indices'] = result.apply(lambda x: x[1])
    
    df['seq_non_dup'] = df['seq_list'].apply(join_list)

    max_length = df['seq_list'].apply(lambda x: len(x)).max()
    print("Maximum length of lists in seq_list column:", max_length)

    empty_lists_exist = any(df['seq_list'].apply(lambda x: len(x) == 0))
    if empty_lists_exist:
        print("There are empty lists in the 'seq_list' column.")
    else:
        print("There are no empty lists in the 'seq_list' column.")

    df['wrong_time_delta_interaction_list'] = df['seq_list'].apply(create_zeros_list)
    df['wrong_time_delta_interaction'] = df['wrong_time_delta_interaction_list'].apply(join_list)

    df.drop(columns=['seq_list', 'item_id_seq_train'], inplace=True)
    df.rename(columns={'seq_non_dup': 'item_id_seq_train'}, inplace=True)

    desired_column_order = ['window_id', 'item_id_seq_train', 'item_id_target', 'wrong_time_delta_interaction', 'session']
    df = df[desired_column_order]
    return df

seq_train_updated = seq_duplicate_remove(seq_train, 'train')
seq_test_updated = seq_duplicate_remove(seq_test, 'test')

# seq_test_updated.to_csv(os.path.join(sequence_path, 'non_dup' ,'test.tsv'), sep='\t', index=False)
# seq_train_updated.to_csv(os.path.join(sequence_path, 'non_dup' ,'train.tsv'), sep='\t', index=False)

Maximum length of lists in seq_list column: 53
There are no empty lists in the 'seq_list' column.
Maximum length of lists in seq_list column: 38
There are no empty lists in the 'seq_list' column.


In [6]:
#Todo to fix the timedelta after deleting the duplicates

# def split_consecutive(numbers):
#     if not numbers:
#         return []

#     # Initialize the list of lists to store consecutive sequences
#     result = [[numbers[0]]]

#     # Iterate through the numbers
#     for num in numbers[1:]:
#         # If the current number is consecutive to the last number in the last sublist
#         if num == result[-1][-1] + 1:
#             result[-1].append(num)  # Add it to the current sublist
#         else:
#             result.append([num])  # Otherwise, start a new sublist

#     return result


# def update_time_delta(row):
#     time_delta_list = row['interaction_time_delta_train_list']
    
#     removed_indices = row['removed_indices']
#     # print(time_delta_list)
#     # print(removed_indices)
#     updated_time_delta = []  # Initialize updated_time_delta variable
    
#     split = split_consecutive(removed_indices)

#     if len(removed_indices) != 0:
#         # for i in range(1, len(time_delta_list)):
#         #     if i not in removed_indices:
#         #         updated_time_delta.append(time_delta_list[i])
#         #     else:
#         #         new_time_delta = time_delta_list[i] + time_delta_list[i-1]
#         #         updated_time_delta.append(new_time_delta)
#         for count, i in enumerate(split):
#             print('count', count)
#             if count == 0:
#                 print('inside count =0', i)
#                 new_time_delta = time_delta_list[i[0]-1]
#             else:
#                 print('i am here')
#                 print(i)
#                 print(new_time_delta)
#                 new_time_delta = updated_time_delta[-1]
#                 print(new_time_delta)
#             for j in i:
#                 print('inside for loop',j)
#                 new_time_delta = new_time_delta + time_delta_list[j]
#             updated_time_delta.append(new_time_delta)
#     else:
#         updated_time_delta = time_delta_list  # Assign original time_delta_list
    
#     return updated_time_delta  # Return the updated list

# seq_train_updated['updated_interaction_time_delta'] = seq_train_updated[['interaction_time_delta_train_list', 'removed_indices']].apply(update_time_delta, axis=1)

In [7]:
seq_train_updated['dup_check'] = seq_train_updated['item_id_seq_train'] + '_' + seq_train_updated['item_id_target'].astype(str)
duplicate_rows = seq_train_updated[seq_train_updated.duplicated(subset=['dup_check'], keep=False)]
duplicate_rows['dup_item_id_seq_train_list'] = duplicate_rows['item_id_seq_train'].str.split().apply(lambda x: [int(i) for i in x])
duplicate_rows['item_id_sum'] = duplicate_rows['dup_item_id_seq_train_list'].apply(lambda x: sum(x))
duplicate_rows = duplicate_rows.sort_values(by=['item_id_sum', 'item_id_target'])
duplicate_rows['dup_id'] = duplicate_rows.groupby('dup_check').ngroup()
# duplicate_rows['dup_id'] = duplicate_rows['dup_id'] - duplicate_rows['dup_id'].min()
duplicate_rows = duplicate_rows.sort_values(by='dup_id')
duplicate_rows.to_csv('duplicate rows.csv')
duplicate_rows

/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_3878/3025388035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_train_updated['dup_check'] = seq_train_updated['item_id_seq_train'] + '_' + seq_train_updated['item_id_target'].astype(str)
/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_3878/3025388035.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicate_rows['dup_item_id_seq_train_list'] = duplicate_rows['item_id_seq_train'].str.split().apply(lambda x: [int(i) for i in x]

,window_id,item_id_seq_train,item_id_target,wrong_time_delta_interaction,session,dup_check,dup_item_id_seq_train_list,item_id_sum,dup_id
1265,1265,1 15 2,7,0 0 0,1401.0,1 15 2_7,"[1, 15, 2]",18,0
1361,1361,1 15 2,7,0 0 0,1565.0,1 15 2_7,"[1, 15, 2]",18,0
1362,1362,1 15,2,0 0,1565.0,1 15_2,"[1, 15]",16,1
1266,1266,1 15,2,0 0,1401.0,1 15_2,"[1, 15]",16,1
1024,1024,1 15,7,0 0,1311.0,1 15_7,"[1, 15]",16,2
...,...,...,...,...,...,...,...,...,...
2957,2957,9,9,0,3953.0,9_9,[9],9,365
2963,2963,9,9,0,3959.0,9_9,[9],9,365
2956,2956,9,9,0,3953.0,9_9,[9],9,365
2955,2955,9,9,0,3953.0,9_9,[9],9,365


In [27]:
def plot_dc(df):
    unique_window_ids = df['window_id'].unique()
    num_subplots = len(df.columns)
    num_unique_ids = len(unique_window_ids)

    fig, axes = plt.subplots(nrows=num_subplots, ncols=num_unique_ids, figsize=(12*num_unique_ids, 6*num_subplots))

    for i, column in enumerate(df.columns):
        for j, window_id in enumerate(unique_window_ids):
            data_subset = df[df['window_id'] == window_id]
            axes[i, j].plot(data_subset.index, data_subset[column])
            axes[i, j].set_title(f"{column} - Window ID {window_id}")
            axes[i, j].set_xlabel('Index')
            axes[i, j].set_ylabel('Values')
            axes[i, j].grid(True)
            axes[i, j].legend([column], loc='upper left')

    plt.tight_layout()
    plt.show()

In [26]:
dc_train.columns

Index(['avg_irradiation', 'steering_speed', 'temperature_out',
       'light_sensor_rear', 'light_sensor_front', 'temperature_in',
       'KBI_speed', 'soc', 'latitude', 'longitude', 'street_category',
       'altitude', 'time_second', 'distance_driven', 'window_id', 'session',
       'datetime'],
      dtype='object')

In [16]:
dc_selected = ['avg_irradiation', 'steering_speed', 'temperature_out',
       'light_sensor_rear', 'light_sensor_front', 'temperature_in',
       'KBI_speed', 'soc', 'latitude', 'longitude', 'street_category',
       'altitude', 'time_second', 'distance_driven']

In [30]:
for dup in sorted(duplicate_rows.dup_id.unique().tolist()):
    row = duplicate_rows[duplicate_rows['dup_id']==dup]
    window_ids = row.window_id.tolist()
    dc_train_curr = dc_train[dc_train['window_id'].isin(window_ids)]
    dc_train_curr = dc_train_curr[dc_selected]
    plot_dc(dc_train_curr)
    break

KeyError: 'window_id'

In [8]:
len(duplicate_rows['window_id'].unique().tolist())

2547

In [9]:
len(duplicate_rows['session'].unique().tolist())

929